In [6]:
import os
import re
import json
import pickle
import requests

from bs4 import BeautifulSoup
from urllib.parse import urljoin


In [7]:
nips_papers = 'https://papers.nips.cc/'
html_text = requests.get(nips_papers).text
soup = BeautifulSoup(html_text, 'html.parser')


In [8]:
def get_num_citations(title, authors, year):
    ### Has issues, check
    ### Has issues, check
    authors = '+'.join(['+'.join([aa['given_name'], aa['family_name']]) for aa in authors])
    req_string = 'https://scholar.google.co.uk/scholar?as_q=&as_epq={title}&as_occt=title&as_sauthors={authors}&as_publication='.format(
        title=title, authors=authors)
    html_text = requests.get(req_string).text
    re_result = re.search('>Cited by (.*?)</a>', html_text)
    num_citations = None if re_result is None else int(re_result.group(1))
    return num_citations

def get_reviews(html_handle):
    reviewer_soup = BeautifulSoup(html_handle, 'html.parser')
    reviewer_dict = {}
    for reviewer in reviewer_soup.find_all('h3'):
        review_text = ''
        for sib in reviewer.find_next_siblings():
            if sib.name == "h3":
                break
            else:
                review_text += ' ' + sib.text
        re_result = re.search('Confidence in this Review (.*?)-', review_text)
        review_conf = None if re_result is None else int(re_result.group(1))
        reviewer_dict[reviewer.contents[0]] = {
            'text': review_text, 'confidence': review_conf}
    return reviewer_dict
    

In [9]:
# Check if there is already some data
if os.path.isfile('neurips_conf_data.pkl'):
    with open('neurips_conf_data.pkl', 'rb') as handle:
        conf_data = pickle.load(handle)
else:
    conf_data = {}


In [24]:

# Loop through all conference years
all_conferences = [cc for cc in soup.find_all('li') if 'paper' in cc.a.get('href')]
all_conferences = all_conferences[::-1]

offset_conf = len(conf_data)
for cc in all_conferences[offset_conf:]:
    conf_link = urljoin(nips_papers, cc.a.get('href'))
    conf_year = conf_link.split('/')[-1]  
    html_text = requests.get(conf_link).text
    conf = BeautifulSoup(html_text, 'html.parser')
    
    # Loop through all current conference's papers
    print("\n\nProcessing: ", cc.a.contents[0])
    paper_list = []
    offset_papers = len(paper_list)
    all_papers = [pp for pp in conf.find_all('li') if 'paper' in pp.a.get('href')]
    
    for pi, pp in enumerate(all_papers[offset_papers:]):
        
        # Get paper info
        print(" - paper [{}/{}]: {}".format(pi + 1 + offset_papers, len(all_papers), pp.a.contents[0]))
        paper_link = urljoin(conf_link, pp.a.get('href')) 
        html_paper = requests.get(paper_link).text
        
        # Extract paper metadata
        if "Metadata" in html_paper:
            link_file = paper_link.replace('hash', 'file')
            link_meta = link_file.replace('html', 'json')
            link_meta = link_meta.replace('Abstract', 'Metadata')
            html_text = requests.get(link_meta).text
            if html_text == 'Resource Not Found':
                conf = BeautifulSoup(html_paper, 'html.parser')
                abstract_text = conf.find('h4', string='Abstract').next_sibling.next_sibling.contents[0]
                abstract = None if abstract_text == 'Abstract Unavailable' else abstract_text
                abstract = abstract.replace('<p>', '')
                abstract = abstract.replace('</p>', '')
                abstract = abstract.replace('\n', ' ')
                author_list = [
                    {'given_name': aa.split(' ')[0],
                     'family_name': aa.split(' ')[1],
                     'institution': None} for aa in pp.i.contents[0].split(', ')]
                paper_meta = {
                    'title': str(pp.a.contents[0]),
                    'authors': author_list,
                    'abstract': abstract,
                    'full_text': None} 
            else:
                paper_meta = json.loads(html_text)
                if 'full_text' in paper_meta.keys():
                    paper_meta['full_text'] = paper_meta['full_text'].replace('\n', ' ')
        else:
            # make nicer not copy
            conf = BeautifulSoup(html_paper, 'html.parser')
            abstract_text = conf.find('h4', string='Abstract').next_sibling.next_sibling.contents[0]
            abstract = None if abstract_text == 'Abstract Unavailable' else str(abstract_text)
            abstract = abstract.replace('<p>', '')
            abstract = abstract.replace('</p>', '')
            abstract = abstract.replace('\n', ' ')
            author_list = [
                {'given_name': aa.split(' ')[0],
                 'family_name': aa.split(' ')[1],
                 'institution': None} for aa in pp.i.contents[0].split(', ')]
            paper_meta = {
                'title': str(pp.a.contents[0]),
                'authors': author_list,
                'abstract': abstract,
                'full_text': None}
       
        # Extract paper supplemental
        if "Supplemental" in html_paper:
            has_supplement = True
        else:
            has_supplement = False
    
        # Extract paper reviews
        if "Reviews" in html_paper:
            link_file = paper_link.replace('hash', 'file')
            link_review = link_file.replace('Abstract', 'Reviews')
            html_text = requests.get(link_review).text
            reviews = get_reviews(html_text)
        else:
            reviews = None
            
        # Extract scholar citation data
        num_cit = get_num_citations(title=paper_meta['title'], authors=paper_meta['authors'], year=conf_year)
        
        # Update paper info
        paper_meta.update({
            'year': conf_year,
            'citations': num_cit,
            'institutions': list(set([aa['institution'] for aa in paper_meta['authors']])),
            'reviews': reviews,
            'has_supplement': has_supplement})
        paper_list.append(paper_meta)
        
        
    # Update conference info and save to file
    conf_data[conf_year] = paper_list
    with open('../data/neurips_conf_data.pkl', 'wb') as handle:
        pickle.dump(conf_data, handle, protocol=pickle.HIGHEST_PROTOCOL)




Processing:  Advances in Neural Information Processing Systems 33  pre-proceedings (NeurIPS 2020)
 - paper [92/1898]: Blind Video Temporal Consistency via Deep Video Prior
 - paper [94/1898]: Simplify and Robustify Negative Sampling for Implicit Collaborative Filtering
 - paper [96/1898]: Model Selection for Production System via Automated Online Experiments
 - paper [98/1898]: On the Almost Sure Convergence of Stochastic Gradient Descent in Non-Convex Problems
 - paper [100/1898]: Automatic Perturbation Analysis for Scalable Certified Robustness and Beyond
 - paper [102/1898]: Adaptation Properties Allow Identification of Optimized Neural Codes
 - paper [104/1898]: Global Convergence and Variance Reduction for a Class of Nonconvex-Nonconcave Minimax Problems
 - paper [106/1898]: Model-Based Multi-Agent RL in Zero-Sum Markov Games with Near-Optimal Sample Complexity
 - paper [108/1898]: Conservative Q-Learning for Offline Reinforcement Learning
 - paper [110/1898]: Online Influence M

 - paper [280/1898]: Cross-lingual Retrieval for Iterative Self-Supervised Training
 - paper [282/1898]: Rethinking pooling in graph neural networks
 - paper [284/1898]: Pointer Graph Networks
 - paper [286/1898]: Gradient Regularized V-Learning for Dynamic Treatment Regimes
 - paper [288/1898]: Faster Wasserstein Distance Estimation with the Sinkhorn Divergence
 - paper [290/1898]: Forethought and Hindsight in Credit Assignment
 - paper [292/1898]: Robust Recursive Partitioning for Heterogeneous Treatment Effects with Uncertainty Quantification
 - paper [294/1898]: Rescuing neural spike train models from bad MLE
 - paper [296/1898]: Lower Bounds and Optimal Algorithms for Personalized Federated Learning
 - paper [298/1898]: Black-Box Certification with Randomized Smoothing: A Functional Optimization Based Framework
 - paper [300/1898]: Deep Imitation Learning for Bimanual Robotic Manipulation
 - paper [302/1898]: Stationary Activations for Uncertainty Calibration in Deep Learning
 - p

 - paper [466/1898]: Breaking the Communication-Privacy-Accuracy Trilemma
 - paper [468/1898]: Audeo: Audio Generation for a Silent Performance Video
 - paper [470/1898]: Ode to an ODE
 - paper [472/1898]: Self-Distillation Amplifies Regularization in Hilbert Space
 - paper [474/1898]: Coupling-based Invertible Neural Networks Are Universal Diffeomorphism Approximators
 - paper [476/1898]: Community detection using fast low-cardinality semidefinite programming 
 - paper [478/1898]: Modeling Noisy Annotations for Crowd Counting
 - paper [480/1898]: An operator view of policy gradient methods
 - paper [482/1898]: Demystifying Contrastive Self-Supervised Learning: Invariances, Augmentations and Dataset Biases
 - paper [484/1898]: Online MAP Inference of Determinantal Point Processes
 - paper [486/1898]: Video Object Segmentation with Adaptive Feature Bank and Uncertain-Region Refinement
 - paper [488/1898]: Inferring learning rules from animal decision-making
 - paper [490/1898]: Input-Aw

 - paper [654/1898]: Low Distortion Block-Resampling with Spatially Stochastic Networks
 - paper [656/1898]: Continual Deep Learning by Functional Regularisation of Memorable Past
 - paper [658/1898]: Distance Encoding: Design Provably More Powerful Neural Networks for Graph Representation Learning
 - paper [660/1898]: Fast Fourier Convolution
 - paper [662/1898]: Unsupervised Learning of Dense Visual Representations
 - paper [664/1898]: Higher-Order Certification For Randomized Smoothing
 - paper [666/1898]: Learning Structured Distributions From Untrusted Batches: Faster and Simpler
 - paper [668/1898]: Hierarchical Quantized Autoencoders
 - paper [670/1898]: Diversity can be Transferred: Output Diversification for White- and Black-box Attacks
 - paper [672/1898]: POLY-HOOT: Monte-Carlo Planning in Continuous Space MDPs with Non-Asymptotic Analysis
 - paper [674/1898]: AvE: Assistance via Empowerment
 - paper [676/1898]: Variational Policy Gradient Method for Reinforcement Learning w

 - paper [840/1898]: Structured Convolutions for Efficient Neural Network Design
 - paper [842/1898]: Latent World Models For Intrinsically Motivated Exploration
 - paper [844/1898]: Estimating Rank-One Spikes from Heavy-Tailed Noise via Self-Avoiding Walks
 - paper [846/1898]: Policy Improvement via Imitation of Multiple Oracles
 - paper [848/1898]: Training Generative Adversarial Networks by Solving Ordinary Differential Equations
 - paper [850/1898]: Learning of Discrete Graphical Models with Neural Networks
 - paper [852/1898]: RepPoints v2: Verification Meets Regression for Object Detection
 - paper [854/1898]: Unfolding the Alternating Optimization for Blind Super Resolution
 - paper [856/1898]: Entrywise convergence of iterative methods for eigenproblems
 - paper [858/1898]: Learning Object-Centric Representations of Multi-Object Scenes from Multiple Views
 - paper [860/1898]: A Catalyst Framework for Minimax Optimization
 - paper [862/1898]: Self-supervised Co-Training for Vide

...

 - paper [3072/1898]: One Ring to Rule Them All: Certifiably Robust Geometric Perception with Outliers
 - paper [3074/1898]: Task-Robust Model-Agnostic Meta-Learning
 - paper [3076/1898]: R-learning in actor-critic model offers a biologically relevant mechanism for sequential decision-making
 - paper [3078/1898]: Revisiting Frank-Wolfe for Polytopes: Strict Complementarity and Sparsity
 - paper [3080/1898]: Fast Convergence of Langevin Dynamics on Manifold: Geodesics meet Log-Sobolev
 - paper [3082/1898]: Tensor Completion Made Practical
 - paper [3084/1898]: Optimization and Generalization Analysis of Transduction through Gradient Boosting and Application to Multi-scale Graph Neural Networks
 - paper [3086/1898]: Content Provider Dynamics and Coordination in Recommendation Ecosystems
 - paper [3088/1898]: Almost Surely Stable Deep Dynamics
 - paper [3090/1898]: Experimental design for MRI by greedy policy search
 - paper [3092/1898]: Expert-Supervised Reinforcement Learning for Offlin

 - paper [3252/1898]: Estimating Training Data Influence by Tracing Gradient Descent
 - paper [3254/1898]: Joint Policy Search for Multi-agent Collaboration with Imperfect Information
 - paper [3256/1898]: Adversarial Bandits with Corruptions: Regret Lower Bound and No-regret Algorithm
 - paper [3258/1898]: Beta R-CNN: Looking into Pedestrian Detection from Another Perspective
 - paper [3260/1898]: Batch Normalization Biases Residual Blocks Towards the Identity Function in Deep Networks
 - paper [3262/1898]: Learning Retrospective Knowledge with Reverse Reinforcement Learning
 - paper [3264/1898]: Dialog without Dialog Data: Learning Visual Dialog Agents from VQA Data
 - paper [3266/1898]: GCOMB: Learning Budget-constrained Combinatorial Algorithms over Billion-sized Graphs
 - paper [3268/1898]: A General Large Neighborhood Search Framework for Solving Integer Linear Programs
 - paper [3270/1898]: A Theoretical Framework for Target Propagation
 - paper [3272/1898]: OrganITE: Optimal tr

 - paper [3440/1898]: Understanding Anomaly Detection with Deep Invertible Networks through Hierarchies of Distributions and Features
 - paper [3442/1898]: Fair Hierarchical Clustering
 - paper [3444/1898]: Self-training Avoids Using Spurious Features Under Domain Shift
 - paper [3446/1898]: Improving Online Rent-or-Buy Algorithms with Sequential Decision Making and ML Predictions
 - paper [3448/1898]: CircleGAN: Generative Adversarial Learning across Spherical Circles
 - paper [3450/1898]: WOR and $p$'s: Sketches for $\ell_p$-Sampling Without Replacement
 - paper [3452/1898]: Hypersolvers: Toward Fast Continuous-Depth Models
 - paper [3454/1898]: Log-Likelihood Ratio Minimizing Flows: Towards Robust and Quantifiable Neural Distribution Alignment
 - paper [3456/1898]: Escaping the Gravitational Pull of Softmax
 - paper [3458/1898]: Regret in Online Recommendation Systems
 - paper [3460/1898]: On Convergence and Generalization of Dropout Training
 - paper [3462/1898]: Second Order Optim

 - paper [3622/1898]: Texture Interpolation for Probing Visual Perception
 - paper [3624/1898]: Hierarchical Neural Architecture Search for Deep Stereo Matching
 - paper [3626/1898]: MuSCLE: Multi Sweep Compression of LiDAR using Deep Entropy Models
 - paper [3628/1898]: Implicit Bias in Deep Linear Classification: Initialization Scale vs Training Accuracy
 - paper [3630/1898]: Focus of Attention Improves Information Transfer in Visual Features
 - paper [3632/1898]: Auditing Differentially Private Machine Learning: How Private is Private SGD?
 - paper [3634/1898]: A Dynamical Central Limit Theorem for Shallow Neural Networks
 - paper [3636/1898]: Measuring Systematic Generalization in Neural Proof Generation with Transformers
 - paper [3638/1898]: Big Self-Supervised Models are Strong Semi-Supervised Learners
 - paper [3640/1898]: Learning from Label Proportions: A Mutual Contamination Framework
 - paper [3642/1898]:  Fast Matrix Square Roots with Applications to Gaussian Processes and

RecursionError: maximum recursion depth exceeded